In [27]:
import pandas as pd

In [28]:
#Loading data
df = pd.read_csv("D:\\archive\\spotify_millsongdata.csv")

In [29]:
df.shape

(57650, 4)

In [30]:
#Kiểm tra các ô null
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [31]:
df = df.drop('link', axis = 1)

In [32]:
df.head(5)

,artist,song,text
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante","Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...


In [33]:
df['text'][0]

"Look at her face, it's a wonderful face  \r\nAnd it means something special to me  \r\nLook at the way that she smiles when she sees me  \r\nHow lucky can one fellow be?  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?  \r\n  \r\nAnd when we go for a walk in the park  \r\nAnd she holds me and squeezes my hand  \r\nWe'll go on walking for hours and talking  \r\nAbout all the things that we plan  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?\r\n\r\n"

In [34]:
df.sample(5000)

,artist,song,text
20230,Uriah Heep,Cry Freedom,She told the story \r\nI listened with my eye...
31923,Europe,Here Comes The Night,Here comes the night \r\nIt's gonna be a long...
24080,Andy Williams,Blue Hawaii,Night and you and blue hawaii \r\nThe night i...
46270,Norah Jones,She's 22,She's twenty two \r\nAnd she's loving you \r...
36121,Hollies,Lucille,"Lucille, you don't do your daddy's will \r\nL..."
...,...,...,...
1649,Bob Marley,Try Me,Try me (try me) \r\nTry me (try me) \r\nTry ...
11643,Linda Ronstadt,Let's Get Together,Love is but a song we sing \r\nFear a way we ...
46546,Offspring,Video Killed The Radio Star,I heard you on my wireless back in '52 \r\nLy...
3870,Dean Martin,Artists And Models (feat. Jerry Lewis),You can trace the mystery of ancient history ...


In [35]:
df = df.sample(5000)

In [36]:
df.head(5)

,artist,song,text
36903,Ingrid Michaelson,Sort Of,Baby you've got the sort of hands to rip me ap...
24498,Backstreet Boys,Larger Than Life,I may run and hide \r\nWhen you're screamin' ...
23558,Alison Krauss,Blue Trail Of Sorrow,Blue trail of sorrow \r\nLies upon the ground...
24083,Andy Williams,Butterfly,"You tell me you love me, you say you'll be tru..."
45374,Neil Young,American Dream,I used to see you \r\nOn every T. V. \r\nYou...


Cleaning

In [37]:
df['text']=df['text'].str.lower().replace('r^\w\s',' ').replace(r'\n',' ', regex=True)

In [38]:
df.head(5)

,artist,song,text
36903,Ingrid Michaelson,Sort Of,baby you've got the sort of hands to rip me ap...
24498,Backstreet Boys,Larger Than Life,i may run and hide \r when you're screamin' m...
23558,Alison Krauss,Blue Trail Of Sorrow,blue trail of sorrow \r lies upon the ground ...
24083,Andy Williams,Butterfly,"you tell me you love me, you say you'll be tru..."
45374,Neil Young,American Dream,i used to see you \r on every t. v. \r your ...


Tokenization và Vectorization

In [39]:
import nltk
from nltk.stem.porter import PorterStemmer


In [40]:
stemmer = PorterStemmer()

In [41]:
def token(txt):
    token = nltk.word_tokenize(txt) # tách chuỗi văn bản thành các từ  riêng lẻ
    a = [stemmer.stem(w) for w in token] # duyệt qua từng từ trong token và chuyển về dạng gốc
    return " ".join(a) # ghép các từ trong list a thành một chuỗi, ngăn cách bởi dấu " "

In [42]:
token("you are beautiful")

'you are beauti'

In [43]:
df['text'].apply(lambda x: token(x))

36903    babi you 've got the sort of hand to rip me ap...
24498    i may run and hide when you 're screamin ' my ...
23558    blue trail of sorrow lie upon the ground and i...
24083    you tell me you love me , you say you 'll be t...
45374    i use to see you on everi t. v. your smile fac...
                               ...                        
50391    mhmmm , mhmmm , owwww , yeahhhhhh ! well i can...
2368     i 'm in the mood to end these empti day ever w...
48582    you pick me up when i am down and on my own yo...
22460    i want a wreck ball i want to break it all i w...
48239    put your faith in what you most believ in two ...
Name: text, Length: 5000, dtype: object

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [45]:
tfid = TfidfVectorizer(analyzer='word', stop_words='english')
# sử dụng từ riêng lẻ trong văn bản để tạo vector, các option khác: char, char_wb
# stopwords: loại bỏ các từ không mang ý nghĩa như "a", "an", "the", "me", ...

In [46]:
matrix = tfid.fit_transform(df['text'])

In [47]:
similer = cosine_similarity(matrix)
## Kết quả là một ma trận đối xứng, giá trị ở hàng thứ i và cột thứ j thể hiện độ tương đồng cosin giữa hai vector biểu diễn văn bản i và j trong tập dữ liệu.

In [48]:
similer[0]

array([1.        , 0.03153122, 0.06696515, ..., 0.        , 0.0097988 ,
       0.02704516])

In [49]:
import pickle
pickle.dump(similer,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))